In [265]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import keras
import tensorboard
from datetime import datetime

print("Tensorflow version : ", tf.__version__)

tf.random.set_seed(42)

from tensorflow.python.client import device_lib
if tf.config.list_physical_devices('GPU'): 
  # Check GPU support 
  if tf.test.is_built_with_cuda(): 
      os.system("nvidia-smi")
  print(device_lib.list_local_devices())

Tensorflow version :  2.17.0


In [266]:
class SimpleModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.a_variable = tf.Variable(5.0, name="train_me")
    self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
  def __call__(self, x):
    return self.a_variable * x + self.non_trainable_variable

simple_module = SimpleModule(name="simple")

simple_module(tf.constant(5.0))

<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

In [267]:
# All trainable variables
print("trainable variables:", simple_module.trainable_variables)
# Every variable
print("all variables:", simple_module.variables)

trainable variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
all variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'do_not_train_me:0' shape=() dtype=float32, numpy=5.0>)


In [268]:
class Dense(tf.Module):
  def __init__(self, in_features, out_features, name=None):
    super().__init__(name=name)
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def __call__(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [269]:
class SequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a model!
my_model = SequentialModule(name="the_model")

# Call it, with random results
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[0.        1.2104536]], shape=(1, 2), dtype=float32)


In [270]:
print("Submodules:", my_model.submodules)

Submodules: (<__main__.Dense object at 0x0000022325C23C50>, <__main__.Dense object at 0x0000022326BD49B0>)


In [271]:
for var in my_model.variables:
  print(var, "\n")

<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.3274685, -0.8426258,  0.3194337],
       [-1.4075519, -2.3880599, -1.0392479],
       [-0.5573232,  0.539707 ,  1.6994323]], dtype=float32)> 

<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[ 0.08422458, -0.86090374],
       [ 0.37812304, -0.00519627],
       [-0.49453196,  0.6178192 ]], dtype=float32)> 



In [272]:
class FlexibleDenseModule(tf.Module):
  # Note: No need for `in_features`
  def __init__(self, out_features, name=None):
    super().__init__(name=name)
    self.is_built = False
    self.out_features = out_features

  def __call__(self, x):
    # Create variables on first call.
    if not self.is_built:
      self.w = tf.Variable(
        tf.random.normal([x.shape[-1], self.out_features]), name='w')
      self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
      self.is_built = True

    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [273]:
# Used in a module
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = FlexibleDenseModule(out_features=3)
    self.dense_2 = FlexibleDenseModule(out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

my_model = MySequentialModule(name="the_model")
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[4.2893534 0.       ]], shape=(1, 2), dtype=float32)


In [274]:
chkp_path = "ckpt/my_checkpoint"
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)

'ckpt/my_checkpoint'

In [275]:
!dir ckpt

 D 드라이브의 볼륨: Home
 볼륨 일련 번호: C446-298C

 D:\Home\Project\MachineLearning\Tensorflow\Basic\ckpt 디렉터리

2024-10-10  오후 02:32    <DIR>          .
2024-10-10  오후 02:31    <DIR>          ..
2024-10-03  오후 04:02                14 .gitignore
2024-10-10  오후 02:32               459 my_checkpoint.data-00000-of-00001
2024-10-10  오후 02:32               362 my_checkpoint.index
               3개 파일                 835 바이트
               2개 디렉터리  1,315,325,644,800 바이트 남음


In [276]:
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense_1/b/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('model/dense_1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('model/dense_2/b/.ATTRIBUTES/VARIABLE_VALUE', [2]),
 ('model/dense_2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 2])]

In [277]:
new_model = MySequentialModule()
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore("ckpt/my_checkpoint")

# Should be the same result as above
new_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[4.2893534, 0.       ]], dtype=float32)>

In [278]:
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  @tf.function
  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a model with a graph!
my_model = MySequentialModule(name="the_model")

In [279]:
print(my_model([[2.0, 2.0, 2.0]]))
print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

tf.Tensor([[4.1463118 0.       ]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[[4.1463118 0.       ]
  [4.1463118 0.       ]]], shape=(1, 2, 2), dtype=float32)


In [280]:
# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace
# Otherwise the summary will not see the graph.
new_model = MySequentialModule()

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True)
#tf.profiler.experimental.start(logdir)

# Call only one tf.function when tracing.
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

tf.Tensor([[0.        1.0405582]], shape=(1, 2), dtype=float32)


In [281]:
tf.saved_model.save(my_model, "the_saved_model")

INFO:tensorflow:Assets written to: the_saved_model\assets


INFO:tensorflow:Assets written to: the_saved_model\assets


In [282]:
!dir the_saved_model

 D 드라이브의 볼륨: Home
 볼륨 일련 번호: C446-298C

 D:\Home\Project\MachineLearning\Tensorflow\Basic\the_saved_model 디렉터리

2024-10-10  오후 02:32    <DIR>          .
2024-10-10  오후 02:31    <DIR>          ..
2024-10-03  오후 04:02                14 .gitignore
2024-10-08  오후 03:55    <DIR>          assets
2024-10-10  오후 02:32                57 fingerprint.pb
2024-10-10  오후 02:32            17,645 saved_model.pb
2024-10-10  오후 02:32    <DIR>          variables
               3개 파일              17,716 바이트
               4개 디렉터리  1,315,325,480,960 바이트 남음


In [283]:
!dir the_saved_model\variables

 D 드라이브의 볼륨: Home
 볼륨 일련 번호: C446-298C

 D:\Home\Project\MachineLearning\Tensorflow\Basic\the_saved_model\variables 디렉터리

2024-10-10  오후 02:32    <DIR>          .
2024-10-10  오후 02:32    <DIR>          ..
2024-10-10  오후 02:32               490 variables.data-00000-of-00001
2024-10-10  오후 02:32               356 variables.index
               2개 파일                 846 바이트
               2개 디렉터리  1,315,325,480,960 바이트 남음


In [284]:
new_model = tf.saved_model.load("the_saved_model")

In [285]:
isinstance(new_model, SequentialModule)

False

In [286]:
print(my_model([[2.0, 2.0, 2.0]]))
print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

tf.Tensor([[4.1463118 0.       ]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[[4.1463118 0.       ]
  [4.1463118 0.       ]]], shape=(1, 2, 2), dtype=float32)


In [287]:
class MyDense(tf.keras.layers.Layer):
  # Adding **kwargs to support base Keras layer arguments
  def __init__(self, in_features, out_features, **kwargs):
    super().__init__(**kwargs)

    # This will soon move to the build step; see below
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def call(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

simple_layer = MyDense(name="simple", in_features=3, out_features=3)

In [288]:
simple_layer(tf.convert_to_tensor([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[4.1687245, 0.       , 0.       ]], dtype=float32)>

In [289]:
class FlexibleDense(tf.keras.layers.Layer):
  # Note the added `**kwargs`, as Keras supports many arguments
  def __init__(self, out_features, **kwargs):
    super().__init__(**kwargs)
    self.out_features = out_features

  def build(self, input_shape):  # Create the state of the layer (weights)
    self.w = tf.Variable(
      tf.random.normal([input_shape[-1], self.out_features]), name='w')
    self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

  def call(self, inputs):  # Defines the computation from inputs to outputs
    return tf.matmul(inputs, self.w) + self.b

# Create the instance of the layer
flexible_dense = FlexibleDense(out_features=3)

In [290]:
flexible_dense.variables

[]

In [291]:
# Call it, with predictably random results
print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0], [3.0, 3.0, 3.0]])))

Model results: tf.Tensor(
[[2.325138  3.6537764 2.4396515]
 [3.4877071 5.4806643 3.6594777]], shape=(2, 3), dtype=float32)


In [292]:
try:
  print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0, 2.0]])))
except tf.errors.InvalidArgumentError as e:
  print("Failed:", e)

Failed: Exception encountered when calling FlexibleDense.call().

{{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [1,4], In[1]: [3,3] [Op:MatMul] name: 

Arguments received by FlexibleDense.call():
  • inputs=tf.Tensor(shape=(1, 4), dtype=float32)


In [293]:
@keras.saving.register_keras_serializable()
class MySequentialModel(tf.keras.Model):
  def __init__(self, name=None, **kwargs):
    super().__init__(**kwargs)

    self.dense_1 = FlexibleDense(out_features=3)
    self.dense_2 = FlexibleDense(out_features=2)
  def call(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a Keras model!
my_sequential_model = MySequentialModel(name="the_model")

# Call it on a tensor, with random results
print("Model results:", my_sequential_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[4.209519  4.2030063]], shape=(1, 2), dtype=float32)


In [294]:
inputs = tf.keras.Input(shape=[3,])

x = FlexibleDense(3)(inputs)
x = FlexibleDense(2)(x)

my_functional_model = tf.keras.Model(inputs=inputs, outputs=x)

my_functional_model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flexible_dense_51               │ (None, 3)              │             0 │
│ (FlexibleDense)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flexible_dense_52               │ (None, 2)              │             0 │
│ (FlexibleDense)                 │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [295]:
my_functional_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-1.7749081, -0.6990162]], dtype=float32)>

In [296]:
my_sequential_model.save("saved/exname_of_file.keras")
reconstructed_model = tf.keras.models.load_model("saved/exname_of_file.keras")
reconstructed_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-5.7754292,  1.2068138]], dtype=float32)>